In [ ]:
from math import log10
import os

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.plotly as py
import plotly.graph_objs as go

from context import data

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(data["nodes"])
G.add_edges_from(data["edges"])

pos = graphviz_layout(G, prog="dot")

In [ ]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YIOrRd',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=12,
            title='Subgenres',
            titleside='right',
            xanchor='left',
            tickvals=[0.0, 1.0],
            ticktext=['External', '0'],
            ticks='outside'
        ),
        line=dict(width=2)
    )
)

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)

keys = data["subgenres"].keys()
for node in G.adjacency():
    num_children = 0
    children = []
    if node[0] in keys:
        num_children = data["subgenres"][node[0]]["num_children"]
        children = data["subgenres"][node[0]]["children"]
        node_trace['marker']['color'].append(log10(num_children + 1) * 10 + 10)
    else:
        node_trace['marker']['color'].append(0)
    node_info = node[0].title() + "<br>" + str(num_children) + " subgenres"
    if 0 < num_children < 20:
        node_info += "<br> + "
        node_info += "<br> + ".join(sorted(children))
    node_trace['text'].append(node_info)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='<br>Music Genre Network Graph',
        titlefont=dict(size=16),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)
